# (Kohavi, Deng and Vermeer, 2022) A/B Testing Intuition Busters: Common Misunderstandings in Online Controlled Experiments

[Link to Paper](https://drive.google.com/file/d/1oK2HpKKXeQLX6gQeQpfEaCGZtNr2kR76/view?usp=sharing)

## Abstract

Have observed seemingly intuitive concepts being touted, including by A/B tool vendors and agencies, which are misleading, often badly so. Provide recommendations that designers can implement to make it harder for experimenters to make these intuitive mistakes

## 1. Introduction

(very similar to Abstract)

## 2. Motivating Example

*GuessTheTest* is a website that shares "money-making A/B test case studies".

An example shows of a test that ran for 35 days and split 50%/50% for maximum statistical power. The results showed a 337% lift, with:
- Control: 82 visitors, 3 conversions
- Treatment: 75 visitors, 12 conversions

The analysis reported a $p$-value of 0.009 although Fisher's Exact Test would have been more appropriate due to the sapmle size (would be 0.013), and the article stated the result was "far below the standard < 0.05 cutoff" and with observed power of "97%, well beyond the accepted 80% minimum".

The paper claims that the result should not be trsuted and that experiment was underpowered.

## 3. Surprising Results Require Strong Evidence - Lower $p$-values

A very common misunderstanding is that a statistically significant result with $p$-value 0.05 has a 5% chance of being false positive. A common alternative to $p$-values used by commercial vendors is "confidence", which is defined as $(1-p\text{-value})*100\%$, and often misinterpreted as the probability that the result is a true positive.

Then what is the $p$-value? The $p$-value is the probability of obtaining a result equal to or more extreme than what was observed, assuming that *all* the modeling assumptions, including the null hypothesis, $H_0$, are true (Greenland, Senn, et al. 2016). Conditioning on the null hypothesis is critical and most often misunderstood. In probabalistic terms, we have

$$p\text{-value}=P(\Delta\text{ observed or more extreme}|H_0\text{ is true})$$

This conditional probability is not what is being described by many misinterpreters. Many people use variations of the opposite conditional probability: what is the probability of the null hypothesis given the delta observed:

$$P(H_0\text{ is true}|\Delta\text{ observed})$$

Bayes Rule can be used for inverting between these two, but the crux of the problem is that it requires the prior probability of the null hypothesis, although we hardly ever have a valid value for this prior. (Although in companies running A/B tests at scale, can construct good prior estimates based on historical experiments)

One useful metric to look at is the False Positive Risk (FPR), which is the probability that the statistically significant result is a false positive, or the probability that $H_0$ is true (no real effect) when the test was statistically significant. Using the following terminology:
- SS is a statistically significant result
- $\alpha$ is the treshold used to determine SS, commonly 0.05 for a two-tailed $t$-test
- $\beta$ is the type-II error (usually 0.2 for 80% power)
- $\pi$ is the prior probability of the null, that is, $P(H_0)$

Using Bayes rule, we can derive the following:

$$P(H_0|SS)=P(SS|H_0)\frac{P(H_0)}{P(SS)}$$

$$=\frac{P(SS|H_0)P(H_0)}{P(SS|H_0)P(H_0)+P(SS|\lnot H_0)P(\lnot H_0)}$$

$$=\frac{\alpha\pi}{\alpha\pi + (1-\beta)(1-\pi)}$$

Several estimates of historical success rates have been published:
- Microsoft: 33%
- Bing: 15%
- Airbnb Search: 8%

Also the implied FPR:
- Microsoft: 5.9%
- Bing: 15%
- Airbnb Search: 26.4%

Remember, the lower the FPR the better. A higher FPR means that a lot of the statistically significant results you got were actually false.

The above was calculted using:
- $\pi=1$-success rate
- experiments were powered at 80%
- using a $p$-value of 0.05 but plugging in 0.025 because only SS improvements are considered successful in two-tailed $t$-tests.

In practice, some results will have a significantly lower $p$-value than the threshold, meaning lower FPR. FPR can be increased by factors such as multiple hypothesis testing.

If the result of an experiment is highly unusual or surprising, one should invoke Twyman's law - any figure that looks interesting or different is usually wrong - and only accept the result if the $p$-value is very low.

As seen in the next section, the earlier example of 300+% lift has a pre-experiment power of 3% (highly underpowered). Even with a generous success rate, the FPR is 63%. Alternatively, if we want the false positive probability $P(H_0|SS)$ to be 0.05, we would need to set the $p$-value threshold to be 0.0016, much lower than the reported 0.009.

This FPR value can be used to set $\alpha$.

## 4. Experiments with Low Statistical Power are NOT Trustworthy

Statistical power is the probability of detecting a meaningful difference between the variants when there really is one, that is, rejecting the null when there is a true difference of $\delta$.

With an industry standard power of 80%, and $p$-value threshold of 0.05, the sample size for each of two equally sized variants can be determined by this simple formula (van Belle 2002):
$$n=\frac{16\sigma^2}{\delta^2}$$

where
- $n$ is the # of users in each variant - variants are assumed to be of equal size
- $\sigma^2$ is the variance of the metric of interest
- $\delta$ is the sensitivity, or the minimum amount of change you want to detect

The derivation of the formula is useful for this and next section, thus is summarized below:

Given two variants of size $n$ each with a standard deviation of $\sigma$, we reject the null hypothesis that there is no distance between Control and Treatment if the observed value is larger than the $Z_{1-\alpha/2}\times\text{SE}$. The Standard Error for the difference is $\sigma\sqrt{2/n}$.

Similarly, we reject the alternative hypothesis that the diffrence is $\delta$ if the observed value is smaller than $Z_{1-\beta}\times\text{SE}$ from $\delta$.

The critical value is, therefore, when these two rejection criteria are equal:

$$Z_{1-\alpha/2}*\text{SE}=Z_{1-\beta}*\text{SE}$$

Further derivation of this will lead to the above formula.

Also shows how the experiment example above is meaningless because the power is so low (at 3%).

Gelman et al. (2014) show that when power goes below 0.1, the probability of getting the sign wrong (concluding that the effect is positive when it is negative) approaches 50% rapidly.

The general guidance is that A/B tests are useful to detect effects of reasonable magnitudes when you have at least thousands of active users, preferably tens of thousands (Kohavi, Deng and Frasca, et al. 2013)

Ioannidis (2005) made this point in a highly cited paper: *Why Most Published Research Findings Are False*.

In a paper by Button et al. (2013) it is shown that in an analysis of 730 studies, the median statistical power in neuroscience is conservatively estimated at 21%. With such low power, many false positive results are to be expected, and many true effects are likely to be missed.

The Open Science Collaboration (2015) attempted to replicate 100 studies from 3 major psychology journals, where studies typically have low power. Of these, only 36% had significant results compared to 97% in original studies.

When the power is low, the probability of detecting a true effect is small, but another consequence of low power, which is often unrecognized, is that a statistically significant finding with low power is likely to highly exaggerate the size of the effect. The winner's curse says that the "lucky" experimenter who finds an effect in a low power setting, or through repeated tests, is cursed by finding an inflated effect (Lee and Shen 2018, Zollner and Pritchard 2007, Deng et al., 2021). For studies in neuroscience, where power is usually in the range of 8% to 31%, initial treatment effects found are estimated to be inflated by 25% to 50% (Button et al. 2013).

Gelman and Carlin (2014) show that when power is below 50%, the exaggeration ratio, defined as the expectation of the absolute value of the estimate, divided by the true effect size, becomes so high as to be meaningless.





## 5. Post-hoc Power Calculations are Noisy and Misleading

Once you have the observed treatment effect $\delta$, you can plug it into the formula to compute the "post-hoc power":

$$Z_{1-\beta}*\text{SE}=\delta-Z_{1-\alpha/2}*\text{SE}$$
$$Z_{1-\beta}=\delta/\text{SE}-Z_{1-\alpha/2}$$
$$1-\beta=\Phi(\delta/\text{SE}-Z_{1-\alpha/2})$$

The term $\delta/\text{SE}$ is the observed $Z$-value used for the test statistic. It is hence $Z_{1-p\text{-val}/2}$, hence the ad-hoc power is
$$1-\beta=\Phi(Z_{1-p\text{-val}/2}-Z_{1-\alpha/2})$$

This means that the power is fully determined by the $p$-value and $\alpha$.

However, calculating power post-results is misleading. Gelman (2019) wrote that "using observed estimate of effect size is too noisy to be useful".

Greenland (2013) wrote: "for a study as completed (observed), it is analogous to giving odds on a horse race after seeing the outcome" and "post hoc power is unsalvageable as an analytic tool, despite any value it has for study planning."

Given the strong evidence that post-hoc power is a noisey and misleading tool, experimentation systems should not show it at all.

## 6. Minimize Data Processing Options in Experimentation Platforms

Optimizely's initial A/B system was showing near-real-time results, so their users peeked at the data and chose to stop when it was statistically significant, a procedure recommended by the company at the time. This type of multiple testing significantly inflates the type-1 error rates (Johari, et al. 2017).

Flexibility in data collection analysis, and reporting dramatically increases actual false-positive rates (Simmons, Nelson and Simonsohn 2011). The culprit is researcher *degrees of reedom*, which include:
1. Should more data be collected, or should we stop now?
2. Should some observations be excluded (e.g., outliers, bots?)
3. Segmentation by variables (e.g., gender, age, geography) and reporting just those as statistically significant.

Even without intentional $p$-hacking, researchers make multiple choices that lead to a multiple-comparison problem and inflate type-I errors.

For experimentation systems, data processing should be standardized. If there is a reason to modify the standard process, for example, outlier removal, it should be pre-specified as part of the experiment configuration and there should be an audit trail of changes to the configratuion, as is done at Booking.com.

## 7. Beware of Unequal Variants

In theory, a single control can be shared with several treatments, and the theory says that a larger control will be beneficial to reduce the variance (Tang, et al. 2010). Assuming equal variance, the effective sample size of a two-sample test is the harmonic mean
$$\frac{1}{\left(\frac{1}{N_T}+\frac{1}{N_C}\right)}$$

When there is one control taking a proportion $x$ of users and $k$ equally sized treatments with size $\frac{1-x}{k}$, the optimal control size should be chosen by minimizing the sum $\frac{k}{1-x}+\frac{1}{x}$. We diffrentiate to get
$$\frac{k}{(1-x)^2}-\frac{1}{x^2}$$

The optimal control proportion $x$ is the positive solution to
$$(k-1)x^2+2x-1=0$$, which is
$$\frac{1}{\sqrt{k}+1}$$

So, for example, when $k=3$, instead of using 25% of users for all 4 variants, we could use 36.6% for control and 21.1% for the treatments, making control more than 1.5x larger. When $k=9$, control would get 25% and each treatment only 8.3%, making control 3 times the size of treatment.

As mentioned in Kohavi et al (2012), when distributions are skewed, in an unequal assignment, the $t$-test cannot maintain the nominal Type-I error rate on both tails. When a metric is positively skewed, and the control is larger than the treatment, the $t$-test will over-estimate the Type-I error on one tail and under-estimate on the other tail because the skewed distribution convergence to normal is different. But when equal sample sizes are used, the convergence is similar and the $\Delta$ (observed delta) is represented well by a Normal or $t$-distribution.

Two common sources of skewness are
1. heavy-tailed measurements such as revenue and counts, often zero-inflated at the same time
2. binary/conversion metric with very small positive rate

Skewness of a metric decreases with the rate of $\sqrt{n}$ as the sample size increases. Kohavi, Deng, et al. (2014) recommended that sample sizes for each variant large enough such that the skewness of metrics be no greater than $1/\sqrt{355}=0.053$.

For experiment ramp-up, where the focus is to reject at the left tail so we can avoid degredation of experiences to users, using a much larger control can lead to higher-than-expected false rejections, so a correction should be applied. In general, recommend against the use of a large (shared) control.